다양한 단어의 표현방법이 있다.
크게 국소표현방법(Local Representation) , 분산표현방법(Distributed Representation)이 있다.
국소표현방법은 단어 하나를 바로 매핑하는 것으로 인덱스로 매핑하는 방법이 가장 큰 예시이다.
분산표현방법은 단어의 앞 뒤 단어를 기반으로 단어를 정의한다. 즉 뉘앙스를 표현할 수 있게된다.

단어의 등장 순서를 고려하지 않고 오로지 빈도수에 의존하는 국소 표현 방법중 예시는 Bag-of-word가 있다 

이번엔 딥러닝에서 중요하다고 판단되고, 내가 공부했었지만 까먹었던 내용을 정리한다.
기울기 폭주(exploding) 소실(vanishing) 문제이다.

깊은 인공신경망을 학습시키다보면 역전파 과정에서 입력층에 가까워질 수록 기울기가 점차적으로 작아지는 경향이 있다.
입력층에 가까운 층들에서 가중치 업데이트가 잘 되지 않는다면 최적의 모델을 찾기 어렵다.
또한 기울기가 점점 커지더니 결국 비정상적으로 큰 값이 되는 경우도 존재한다.
이런 문제점은 RNN모델에서 흔히 나타나는 문제이기도 하다.

이런 문제를 해결하기 위한 방법은 무엇이 있을까?

1. ReLU와 ReLU 변형 (activation function)
시그모이드 함수의 경우 절대값이 클수록 기울기가 0에 가까워진다. 이러면 기울기 소실 문제가 발생할 가능성이 높다. 
이 문제점을 해결하기 위한 방법으로는 activation function을 sigmoid나 tahn 을 쓰는 대신에 ReLU를 사용하는 방법이 있다.
은닉층에서는 시그모이드보다 ReLU나 Leaky ReLU를 사용하도록 하자.

2. Gradient Clipping
이 방법은 내가 두번째 논문을 작성할 때 사용했던 방법이다. 그래디언트 클리핑이란 말 그대로 기울기 값을 자르는 것에 있다. 기울기 폭주를 막기위해서 임계값을 넘지않게 값을 자른다.
다시 말해서 커지지 않도록 임계값 크기를 감소시키는 방법이다. 
from tensorflow.keras import optimizers

Adam = optimizers.Adam(lr=0.0001, clipnorm=1.)
와 같이 사용가능하다.

3. 가중치 초기화 (weight initialization)
같은 모델을 학습시키더라도 가중치가 초기에 어떤 값을 가지는지가 중요하다. 적절한 가중치로 초기화 하는 것이 기울기 소실과 같은 문제를 해결할 수 있다.

- Xavier init- (glorot init-)
세이비어 초기화라고도 불린다. 이전층의 뉴런갯수와 다음층의 뉴런갯수를 바탕으로 식을 세우며 균등분포나 정규분포를 사용한다.
이 방법은 기울기 분산사이에 균형을 맞춰줘서 특정 층이 주목받는 것을 방지해줄 수 있다. 
하지만 시그모이드나 하이퍼볼릭탄젠트 함수처럼 S자 activation function과는 조합이 좋지만 ReLU와는 성능 조합이 좋지 않다.

-He init-
이 방법은 이전층의 뉴런갯수를 사용해서 분포를 내지만 다음층의 뉴런갯수는 고려하지 않는다.
ReLU계열 함수와 조합이 좋다.

4. 배치 정규화 (Batch Normalization)
activation function과 가중치 초기화로 해결할 수 있지만, 학습과정중에 언제든지 다시 기울기 폭주나 소실이 발생할 수 있다.
배치 정규화는 각 층에 들어가는 입력을 평균과 분산으로 정규화해서 학습을 효율적으로 만드는 방법이다.
기존에는 층마다 데이터의 분포가 바뀌지만 (내부 공변량 변화라고 한다) 배치정규화를 사용하면 분표변화를 줄여줘서 해결할 수 있다고 한다. (내부 공변량 변화가 있으면 기울기 폭주나 소실 문제가 발생한다고 주장)
배치 단위로 정규화가 진행되며, 활성화 함수를 통과하기 전에 수행된다. 
기울기 소실 문제를 크게 개선할 수 있고, 가중치 초기화에 덜 민감하게 된다. 또한 더 큰 학습률을 사용가능하게 함으로서 학습속도를 개선할 수 있다. 
noise를 추가하는 효과 또한 존재하며 실행시간이 살짝 느려질 수 있다. 

하지만 문제점(한계점) 또한 존재한다.
- 미니배치 크기에 의존적이다.
- RNN에 적용하기 어렵다 (time step마다 다른 통계치를 가지기 때문이다)

5. 층 정규화 (Layer Normalization)
배치 정규화는 간단하게 말해서 열끼리 평균과 표준편차를 구하는 방법이라고 한다면 층 정규화는 행별로 평균과 표준편차를 구한다.
이는 미니배치 크기에 의존하지 않고 RNN에도 적용할 수 있다고 한다. 

Keras 훑어보기

앞서 예제를 하면서 Keras를 사용했지만 한번 더 정리하고 넘어가도록 하자.

1. 전처리(Preprocessing)

In [3]:
#Tokenizer()
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
train_text = "The earth is an awesome place live"

#단어 집합 생성 (토크나이저 진행)
tokenizer.fit_on_texts([train_text])

#정수 형태로 표현
sub_text = "The earth is an awesome place live"
sequence = tokenizer.texts_to_sequences([sub_text])[0]

print('정수 인코딩 결과',sequence)
print('토크나이저 결과',tokenizer.word_index)

정수 인코딩 결과 [1, 2, 3, 4, 5, 6, 7]
토크나이저 결과 {'the': 1, 'earth': 2, 'is': 3, 'an': 4, 'awesome': 5, 'place': 6, 'live': 7}


In [5]:
#각 샘플의 길이는 다를 수 있다. 모델의 입력으로 사용하려면 길이를 맞춰줘야하므로 pad_sequences를 사용해서 맞춰주도록 하자
print('padding 결과',pad_sequences([[1,2,3],[3,4,5,6],[7,8]],maxlen=3,padding='pre'))
#maxlen은 패딩의 길이를 얼마나 할 것인가를 나타내고 padding에 pre를 넣으면 앞에 0, post를 넣으면 뒤에 0을 넣는다.

padding 결과 [[1 2 3]
 [4 5 6]
 [0 7 8]]


워드 임베딩(Word embedding)
뒤에서 한번 자세하게 볼건데 워드임베딩이란 텍스트 내 단어들을 밀집 벡터로 만드는 것을 의미한다. 원-핫 인코딩(벡터)는 대부분이 0이고 몇개만 1이였다. 또한 단어벡터들끼리 유사도를 구하기 까다롭다는 단점도 있다.
하지만 워드 임베딩은 상대적으로 저차원의 벡터를 가지며 모든 원소의 값이 실수이다. 표현되는 숫자들은 학습된 숫자이다.
Embedding()은 단어를 밀집벡터로 만들어주며 인공신경망 용어로 임베딩 층을 만든다고 한다. 

In [8]:

#위에 실습에서 진행한 것을 사용하도록 하자.
padded = pad_sequences([[1,2,3],[3,4,5,6],[7,8]],maxlen=3,padding='pre')

from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
vocap_size = len(tokenizer.word_index)+1
embedding_dim = 4
 
model = Sequential()
model.add(Embedding(vocap_size,embedding_dim,input_length = 3))
#이런 느낌으로 선언하고 나중에 fit할 때 패딩된 데이터를 넣으면 된다.

In [10]:
#모델링(Modeling)
#Sequential - 케라스에서 층을 구성하기 위해서 사용한다. Sequential()로 선언 후 add를 통해서 층을 추가한다. 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(1,input_dim=3, activation='relu')) #첫번째 인자 출력층, 두번째 인자 입력 뉴런 수 (입력의 차원), 세번째 층 활성화 함수

print(model.summary())

#이외로 .compile을 통해서 최적화함수, loss함수를 설정할 수 있고, metric을 뭐 쓸건지 설정할 수도 있다.
#추가로 .fit을 통해서 epochs, batch_size, validation_data, validation_split 등을 설정할 수 있다.
#validation_data 는 (v_train, v_test)와 같이 데이터를 직접 넣을 수 있고 validation_split은 0.2로 설정하면 train data의 20%를 검증 데이터로 사용하는 의미가 된다. 
#verbose인자는 학습 중 출력되는 문구를 나타낸다. 0은 출력 x 1은 진행도를 보여주고 2는 미니배치마다 loss값을 나타낸다.

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 1)                 4         
                                                                 
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
None


Sequential()은 사용하기 간단하지만 복잡한 모델을 설계하기 무리가 있다.
따라서 funtional API를 사용해서 모델을 설계해보도록 하자.

In [11]:
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model

X = [1,2,3,4,5,6,7,8,9]
Y = [11,22,33,44,53,66,77,87,95]

input = Input(shape=(1,))
output = Dense(1,activation='linear')(input)
linear_model = Model(input,output)

sgd = optimizers.SGD(lr = 0.01)
linear_model.compile(optimizer=sgd,loss='mse',metrics=['mse'])
linear_model.fit(X,Y,epochs=100,verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 351ms/step - loss: 3677.3889 - mse: 3677.3889
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 453.5410 - mse: 453.5410
Epoch 3/100
1/1 [==============================] - 0s 5ms/step - loss: 56.8699 - mse: 56.8699
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 8.0615 - mse: 8.0615
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 2.0548 - mse: 2.0548
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 1.3147 - mse: 1.3147
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 1.2225 - mse: 1.2225
Epoch 8/100
1/1 [==============================] - 0s 6ms/step - loss: 1.2101 - mse: 1.2101
Epoch 9/100
1/1 [==============================] - 0s 5ms/step - loss: 1.2075 - mse: 1.2075
Epoch 10/100
1/1 [==============================] - 0s 6ms/step - loss: 1.2061 - mse: 1.2061
Epoch 11/100
1/1 [==============================] - 0s 5ms/step -

In [13]:
linear_model.predict([5])
#5시간 하면 54.34점을 받는다고 예측하고 있다. 나름 정확하다

1/1 [==============================] - 0s 90ms/step


array([[54.339893]], dtype=float32)